# Paso 1. Se importan las librerias

In [23]:
import pandas as pd
import tensorflow as tf
import tensorflow.keras as kr
import numpy as np

from summarytools import dfSummary
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.utils import to_categorical


# Paso 2. Cargar los datos

In [5]:
import pandas as pd

# Carga el archivo Excel
df = pd.read_excel('BDD-Reclamos-Consolidado-Total Reclamos.xlsx')

In [6]:
df.head()

,Reclamos,ipcprin,Edad,DepartamentoDescripcion,Sexo,EstadoCivil,Rentabilidadantesreclamo,Segmentoantesreclamo,Area,Abandono
0,2,3,42,GUATEMALA ...,FEMENINO ...,CASADO ...,9046.941583,BAJO,ATENCION AL CLIENTE,0
1,1,2,36,GUATEMALA ...,FEMENINO ...,SOLTERO ...,4.860650,MASIVO,ATENCION AL CLIENTE,0
2,1,2,48,GUATEMALA ...,MASCULINO ...,SOLTERO ...,1279.018169,MEDIO,ATENCION AL CLIENTE,0
3,1,3,41,ZACAPA ...,MASCULINO ...,CASADO ...,13546.512618,PRIVADO,ATENCION AL CLIENTE,0
4,1,5,50,GUATEMALA ...,MASCULINO ...,CASADO ...,14727.358352,ALTO,EMPRESARIAL,0


In [7]:
dfSummary(df)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,Reclamos[int64],Mean (sd) : 1.3 (0.7)min < med < max:1.0 < 1.0 < 8.0IQR (CV) : 0.0 (1.8),7 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjUuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8qNh9FAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACKUlEQVR4nO3ZPW/aYBSG4XMKoQYim4/WbKxsHRjZu/bHZu3ulf+BZMAWjgkIuVuXqiW2QcnT3NfMOVjoBssvXlWVAe/dp7e+AOA1utde4O59M+s13H+qqqpsOAv89s9Q3b0/nU5/hGE4brI8z/Oduz8RK9q69ovaC8NwvFqtyiiKjnUWZ1kWJEkyTtO0Z2aEilau3vrNzKIoOsZx/Nxgf7/BDPAHHqYggVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhoXvP5ZfL5cHMQndvuuJUVVV5w0uCqLuFWhTFw/l8/jafzzudTufYZEee5zt3fyJW3C3U0+nUDYJgsFwuj7PZbFd3PsuyIEmScZqmPTMj1A/urrd+M7PhcHiM4/i54Xj/phcDWTxMQQKhQgKhQgKhQsLdH6baaHkOyxnsf+Tdhtr2HLbtGay7982s12TW+JLc3KtCzbIsqLv4cDgEZmZFUQSbzWZQd3673T4GQTBYLBY+Go1e6syWZfl5vV5/TdP0i7vndd/bzILJZPI9iqLHBrO23+8P7v7TzBr90fFBvVRV9dfP6xcYIop7cnAz5AAAAABJRU5ErkJggg=="">",0(0.0%)
2,ipcprin[int64],Mean (sd) : 3.2 (2.1)min < med < max:0.0 < 3.0 < 40.0IQR (CV) : 2.0 (1.5),17 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjUuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8qNh9FAAAACXBIWXMAAA9hAAAPYQGoP6dpAAAB4UlEQVR4nO3dQWrCQBSH8TeNlVEhQQTdeYQuXHqIHraH8DCCSMRUGwnTTS2F0mpmSOtfvt/aeWTxYcjmjQshGHDrHv77AYBr9C79wDk3MLN+5Pw6hHCIPAt8+jVU59xgMpk853k+jhm+2+22zrkXYkWqS/+o/TzPx8vl8lAUxbHN4LIs/Wq1Gm82m76ZESqSXHz1m5kVRXGcTqevEfMHEWeAb/iYggRChQRChQRChQRChQRChQRChQRChQRChQRChQRChQRChQRChQRChQRChQRChQRChQRChQRChQRChQRChQRChQRChQRChYSrFlDEaprm0cxy51zsCHZXwcw6DLWqqsfT6fQ0n8+zLMtarQM6Y3cVzjoLta7rnvd+uFgsjrPZbNv2PLur8FWnr34zs9FoFLu3yozdVfjAxxQkECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokdL7IN0XiHQDs/78jNxtq6h0A7P+/LzcbasodAOz/vz9XhVqWpW87eL/fezOzqqr8er0exp6P1TRNz9KuDsLfegsh/PjmfAcwzIY67MlOKQAAAABJRU5ErkJggg=="">",0(0.0%)
3,Edad[int64],Mean (sd) : 38.8 (12.7)min < med < max:18.0 < 36.0 < 93.0IQR (CV) : 17.0 (3.1),72 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjUuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8qNh9FAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACnUlEQVR4nO3cvY7aQBRA4TuwIPMjI0QEHU3KSCkoeYg8bB4hSDwBbSo6JAsZ4WBhjFMlTVhYxuMwd3S+fhYXR0YXe6+pqkoA37VefQHAR7y9+gLuMcb0RKRrefxcVdXJ5fXgdbwN1RjTm0wm3+I4HtucPxwOe2PMd2INg7ehikg3juPxcrk8jUaj/JmDaZpG6/V6nCRJV0QINQA+hyoiIqPRKJ9Op78sjvacXwxehmEKKjR6R605DMXX67Xj8nqgV2Oh1h2GiqKIyrL8kuf5DxGx+epHQJq8o1oPQyIi2+12vNls+pfLpd3ExUGXxocp22Fov98zDOEvhimoQKhQgVChAqFCBUKFCoQKFQgVKhAqVCBUqECoUIFQoQKhQgVChQqEChUIFSoQKlQgVKhAqFCBUKGC9wsobJVl2RGR2Bhj+yfYXeWRIEPNsqxTFMXX+XzebrfbT/8HrAi7q3wTZKjn8/ktiqL+YrHIZ7PZ/tnz7K7yT5Ch/jEYDGz3Vomwu8orDFNQgVChwsOv/hqLzlhyBmfuhlpn0RlLzuDSozuq9aIzlpzBpQ9N/TaLzlhyBpcYpqBC0L+j1lHzESyPXx0j1BvqPoLl8at7hHpDnUewPH5tBqHeUeMRLIOkYwxTUIFQoQKhQgVChQqEChUIFSoQKlQgVKjAD/4N4D0B9wjVMd4TaAahOsZ7As0g1IbwnoBbDFNQgTuqZ9iZdRuheoSdWe8jVI+wM+t9hOohdmb9i2EKKhAqVCBUqECoUIFhKjChvhBDqAEJ+YUYQg1I3RdiVqvVNEmST8aYg+UltETkann27t38Q6GmaRo9+6nH4zESEcmyLNrtdv3/eV7rZ7u6bhun06nW3bgsy06WZZ+Hw+HPVqtVPHv+0d38Ny5ggE8N/zxrAAAAAElFTkSuQmCC"">",0(0.0%)
4,DepartamentoDescripcion[object],1. GUATEMALA 2. QUETZALTENANGO 3. SACATEPEQUEZ 4. ESCUINTLA 5. CHIMALTENANGO 6. IZABAL 7. SUCHITEPEQUEZ 8. SAN MARCOS 9. HUEHUETENANGO 10. RETALHULEU 11. other,"3,137 (84.0%)110 (2.9%)89 (2.4%)78 (2.1%)39 (1.0%)34 (0.9%)30 (0.8%)22 (0.6%)22 (0.6%)20 (0.5%)155 (4.1%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjUuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8qNh9FAAAACXBIWXMAAA9hAAAPYQGoP6dpAAAEVklEQVR4nO3dTU4bWQBG0Qey0mFSlmWJZWQBXkQWm0V4AdkHQtQg0JnQg0AUfjLjXafDOTPIoCzlqoxL74Oz+/

# Paso 3. Preprocesamiendo de Datos

In [8]:
# Imputar Valores Faltantes en Variables Numérica
df['Reclamos'].fillna(df['Reclamos'].mean(), inplace=True)
df['ipcprin'].fillna(df['ipcprin'].mean(), inplace=True)
df['Edad'].fillna(df['Edad'].mean(), inplace=True)
df['Rentabilidadantesreclamo'].fillna(df['Rentabilidadantesreclamo'].mean(), inplace=True)


In [9]:
# Tratamiento de Outliers utilizando el metódo de Winsorizing para limitar los valores extremos

from scipy.stats.mstats import winsorize

df['Reclamos'] = winsorize(df['Reclamos'], limits=[0.05, 0.05])
df['ipcprin'] = winsorize(df['ipcprin'], limits=[0.05, 0.05])
df['Edad'] = winsorize(df['Edad'], limits=[0.05, 0.05])
df['Rentabilidadantesreclamo'] = winsorize(df['Rentabilidadantesreclamo'], limits=[0.05, 0.05])


In [10]:
#One-Hot Encoding para Variables Categóricas
df = pd.get_dummies(df, columns=['Segmentoantesreclamo', 'EstadoCivil', 'Sexo', 'DepartamentoDescripcion', 'Area'], drop_first=True)


In [11]:
# División de Datos en Entrenamiento y Prueba
X = df.drop(['Abandono'], axis=1)  # Variables independientes
y = df['Abandono']  # Variable objetivo

In [12]:
dummy_y = kr.utils.to_categorical(y)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, dummy_y, test_size=0.2, random_state=42)

# Paso 4. Modelado

### Creación Red Neuronal

In [17]:
model= kr.models.Sequential()
model.add(kr.layers.Dense(46,input_dim=46,activation='relu'))
model.add(kr.layers.Dense(2,activation='softmax' ))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
model.fit(X_train, y_train, epochs=25, batch_size=10)

Epoch 1/25
299/299 [==============================] - 1s 2ms/step - loss: 14.0768 - accuracy: 0.8297
Epoch 2/25
299/299 [==============================] - 0s 1ms/step - loss: 3.4170 - accuracy: 0.8384
Epoch 3/25
299/299 [==============================] - 0s 1ms/step - loss: 4.0530 - accuracy: 0.8517
Epoch 4/25
299/299 [==============================] - 1s 2ms/step - loss: 2.1370 - accuracy: 0.8604
Epoch 5/25
299/299 [==============================] - 0s 1ms/step - loss: 2.9718 - accuracy: 0.8494
Epoch 6/25
299/299 [==============================] - 0s 1ms/step - loss: 4.3359 - accuracy: 0.8440
Epoch 7/25
299/299 [==============================] - 0s 2ms/step - loss: 3.7522 - accuracy: 0.8548
Epoch 8/25
299/299 [==============================] - 0s 2ms/step - loss: 3.6396 - accuracy: 0.8484
Epoch 9/25
299/299 [==============================] - 0s 1ms/step - loss: 3.7981 - accuracy: 0.8417
Epoch 10/25
299/299 [==============================] - 1s 2ms/step - loss: 4.8243 - accuracy: 0.846

In [22]:
def create_model(num_layers):
    model= kr.models.Sequential()
    model.add(kr.layers.Dense(46,input_dim=46,activation='relu'))
    
    for _ in range(num_layers):
        model.add(kr.layers.Dense(46, activation='relu'))
        
    model.add(kr.layers.Dense(2, activation='softmax'))  # Ajuste en la cantidad de neuronas aquí
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def train_and_evaluate_model(num_layers, X_train, y_train, X_test, y_test):
    model = create_model(num_layers)
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)
    _, accuracy = model.evaluate(X_test, y_test)
    return accuracy

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

##y_train_one_hot = to_categorical(y_train)
###y_test_one_hot = to_categorical(y_test)

best_accuracy = 0
best_num_layers = 0

for num_layers in range(1, 6):
    accuracy = train_and_evaluate_model(num_layers, X_train_scaled, y_train, X_test_scaled, y_test)
    print(f"Accuracy with {num_layers} layers: {accuracy}")
    
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_num_layers = num_layers

print(f"Best accuracy achieved: {best_accuracy} with {best_num_layers} layers.")

24/24 [==============================] - 0s 1ms/step - loss: 0.3712 - accuracy: 0.8824
Accuracy with 1 layers: 0.8823529481887817
24/24 [==============================] - 0s 2ms/step - loss: 0.3617 - accuracy: 0.8810
Accuracy with 2 layers: 0.8810160160064697
24/24 [==============================] - 0s 3ms/step - loss: 0.3639 - accuracy: 0.8824
Accuracy with 3 layers: 0.8823529481887817
24/24 [==============================] - 0s 1ms/step - loss: 0.3638 - accuracy: 0.8770
Accuracy with 4 layers: 0.8770053386688232
24/24 [==============================] - 0s 4ms/step - loss: 0.3838 - accuracy: 0.8757
Accuracy with 5 layers: 0.875668466091156
Best accuracy achieved: 0.8823529481887817 with 1 layers.


In [24]:
def create_model(num_neurons):
    model= kr.models.Sequential()
    model.add(kr.layers.Dense(46,input_dim=46,activation='relu'))
    model.add(kr.layers.Dense(2,activation='softmax' ))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def train_and_evaluate_model(num_neurons, X_train, y_train, X_test, y_test):
    model = create_model(num_neurons)
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)
    _, accuracy = model.evaluate(X_test, y_test)
    return accuracy

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

##y_train_one_hot = to_categorical(y_train)
##y_test_one_hot = to_categorical(y_test)

start_neurons = 10
end_neurons = 300
num_neurons_list = np.logspace(np.log10(start_neurons), np.log10(end_neurons), num=10, dtype=int)

best_accuracy = 0
best_num_neurons = 0

for num_neurons in num_neurons_list:
    accuracy = train_and_evaluate_model(num_neurons, X_train_scaled, y_train, X_test_scaled, y_test)
    print(f"Accuracy with {num_neurons} neurons: {accuracy}")
    
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_num_neurons = num_neurons

print(f"Best accuracy achieved: {best_accuracy} with {best_num_neurons} neurons.")

24/24 [==============================] - 0s 1ms/step - loss: 0.3589 - accuracy: 0.8797
Accuracy with 10 neurons: 0.8796791434288025
24/24 [==============================] - 0s 1ms/step - loss: 0.3552 - accuracy: 0.8837
Accuracy with 14 neurons: 0.883689820766449
24/24 [==============================] - 0s 1ms/step - loss: 0.3542 - accuracy: 0.8864
Accuracy with 21 neurons: 0.8863636255264282
24/24 [==============================] - 0s 1ms/step - loss: 0.3573 - accuracy: 0.8837
Accuracy with 31 neurons: 0.883689820766449
24/24 [==============================] - 0s 1ms/step - loss: 0.3579 - accuracy: 0.8824
Accuracy with 45 neurons: 0.8823529481887817
24/24 [==============================] - 0s 2ms/step - loss: 0.3536 - accuracy: 0.8824
Accuracy with 66 neurons: 0.8823529481887817
24/24 [==============================] - 0s 2ms/step - loss: 0.3522 - accuracy: 0.8810
Accuracy with 96 neurons: 0.8810160160064697
24/24 [==============================] - 0s 2ms/step - loss: 0.3587 - accuracy

In [25]:
modelfinal= kr.models.Sequential()
modelfinal.add(kr.layers.Dense(21,input_dim=46,activation='relu'))
modelfinal.add(kr.layers.Dense(2,activation='softmax' ))
modelfinal.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [27]:
modelfinal.fit(X_train, y_train, epochs=25, batch_size=10)

Epoch 1/25
299/299 [==============================] - 1s 2ms/step - loss: 4.2222 - accuracy: 0.8414
Epoch 2/25
299/299 [==============================] - 1s 2ms/step - loss: 2.5935 - accuracy: 0.8601
Epoch 3/25
299/299 [==============================] - 1s 2ms/step - loss: 1.9945 - accuracy: 0.8665
Epoch 4/25
299/299 [==============================] - 1s 2ms/step - loss: 1.6943 - accuracy: 0.8678
Epoch 5/25
299/299 [==============================] - 0s 1ms/step - loss: 1.3931 - accuracy: 0.8638
Epoch 6/25
299/299 [==============================] - 0s 2ms/step - loss: 1.5457 - accuracy: 0.8732
Epoch 7/25
299/299 [==============================] - 1s 2ms/step - loss: 2.5019 - accuracy: 0.8624
Epoch 8/25
299/299 [==============================] - 1s 2ms/step - loss: 1.3653 - accuracy: 0.8732
Epoch 9/25
299/299 [==============================] - 0s 2ms/step - loss: 2.3397 - accuracy: 0.8561
Epoch 10/25
299/299 [==============================] - 1s 3ms/step - loss: 1.0556 - accuracy: 0.8722

# Paso 5. Evaluación de Modelo

In [19]:
loss, accuracy = model.evaluate(X_test, y_test)

# Imprimimos la precisión del modelo, multiplicada por 100 para obtener un porcentaje.
print('Precisión: %.2f' % (accuracy*100))

24/24 [==============================] - 0s 820us/step - loss: 3.0524 - accuracy: 0.8837
Precisión: 88.37


In [28]:
loss, accuracy = modelfinal.evaluate(X_test, y_test)

# Imprimimos la precisión del modelo, multiplicada por 100 para obtener un porcentaje.
print('Precisión: %.2f' % (accuracy*100))

24/24 [==============================] - 0s 2ms/step - loss: 0.6369 - accuracy: 0.8837
Precisión: 88.37
